In [65]:
import urllib2
import urllib
from lxml import html
import re
list_o_movies = ['room', 'the-revenant-2015']

mc= 'http://www.metacritic.com/movie/room/critic-reviews'

In [47]:
url = mc
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
values = {'name' : 'Michael Foord',
          'location' : 'Northampton',
          'language' : 'Python' }
headers = { 'User-Agent' : user_agent }

data = urllib.urlencode(values)
req = urllib2.Request(url, data, headers)
response = urllib2.urlopen(req)
the_page = response.read()

In [48]:
tree = html.fromstring(the_page)

In [1]:
# tree.xpath('//a/text()')

In [16]:
#possibilities = re.findall('([^ ]+)="([^"/]*(?=article|body|story).*?[^/]?)"', tree.body,re.IGNORECASE)

In [17]:
path = '//li[@class="review_action full_review"]/a/@href'
links = tree.body.xpath(path)

In [18]:
import BeautifulSoup


In [2]:
r= 'http://www.imdb.com/find?ref_=nv_sr_fn&q=Room&s=all'
url = r
def get_tree(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'name' : 'Michael Foord',
	      'location' : 'Northampton',
	      'language' : 'Python' }
    headers = { 'User-Agent' : user_agent }

    data = urllib.urlencode(values)
    req = urllib2.Request(url, data, headers)
    response = urllib2.urlopen(req)
    the_page = response.read()
    tree = html.fromstring(the_page)	
    return tree

In [39]:
#get all year links 
#get 2016 results from first page
# get all results from each year
oscar_page = 'http://www.imdb.com/awards-central/oscars'
tree = get_tree(oscar_page)
blurbs = "//div[@id='sidebar']//div[@class='aux-content-widget-2']//a"
"//div[@class='ab_ninja']/p[@class='blurb']"
oscar_year_page_links = tree.body.xpath(blurbs)

In [43]:
base = 'http://www.imdb.com'
urls = []
for p in oscar_year_page_links:
    if re.match(r'\d{4}', p.text):

       year = re.findall(r'\d{4}', p.text).pop()

       urls.append((year,base+p.body.xpath('//a[text()="'+year+'"]/@href')[0] ))     

In [75]:
oscar_pages = []
for y, p in urls:
    t = get_tree(p)
    oscar_pages.append((y,t))
    #urllib.urlretrieve(p, './oscar_webpages/oscars_{y}.html')
    #!wget -O oscars_{y}.html {p}
    

In [86]:
t = get_tree('http://www.imdb.com/event/ev0000003/2016')	
oscar_pages.append(("2016",t))

In [418]:
award_titles = []
for a in oscar_pages:

#    Properties[Property[@Descriptor = '2'] = 'FOO']
    award_title_list = a[1].xpath('//div[@class="award"][h1[text()="Oscar"]]/blockquote/h2/text()')

    award_titles.append((a[0], award_title_list))


In [419]:
win_noms = []
for i, a in enumerate(oscar_pages):
    win_nom_blocks = a[1].xpath('//div/div/div[@class="award"][h1[text()="Oscar"]]/blockquote/blockquote')
    win_noms.append((a[0], win_nom_blocks))

In [420]:
print len(award_titles)
print len(win_noms)

88
88


In [487]:

with codecs.open('testoscarout.txt', 'w', encoding='utf-8') as w:
    for i, at in enumerate(award_titles[:1]):
        win_nom_set = win_noms[i]
        year = win_nom_set[0]

        for j, wn_block in enumerate(win_nom_set[1]):
            for k, (movie, person, won) in enumerate(get_win_nom_data(wn_block)):

                w.write(year+ " ||| " +at[1][j]+ " ||| " +movie+ " ||| " + person +" ||| "+ won +"\n")

2015  |||  Best Motion Picture of the Year  |||  Birdman or (The Unexpected Virtue of Ignorance)  |||  Alejandro G. Iñárritu  |||  WON
2015  |||  Best Motion Picture of the Year  |||  Birdman or (The Unexpected Virtue of Ignorance)  |||  John Lesher  |||  WON
2015  |||  Best Motion Picture of the Year  |||  Birdman or (The Unexpected Virtue of Ignorance)  |||  James W. Skotchdopole  |||  WON
2015  |||  Best Motion Picture of the Year  |||  American Sniper  |||  Clint Eastwood  |||  LOST
2015  |||  Best Motion Picture of the Year  |||  American Sniper  |||  Robert Lorenz  |||  LOST
2015  |||  Best Motion Picture of the Year  |||  American Sniper  |||  Andrew Lazar  |||  LOST
2015  |||  Best Motion Picture of the Year  |||  American Sniper  |||  Bradley Cooper  |||  LOST
2015  |||  Best Motion Picture of the Year  |||  American Sniper  |||  Peter Morgan  |||  LOST
2015  |||  Best Motion Picture of the Year  |||  Boyhood  |||  Richard Linklater  |||  LOST
2015  |||  Best Motion Picture of

In [486]:
def get_win_nom_data(wn_block):
    movies = wn_block.xpath('./div')

    info = []

    
    for i, m in enumerate(movies):
    	mov_title = m.xpath('./strong/a/text()')[0]
        won='LOST'
        if i==0:
            won='WON'
	people = m.xpath('./a/text()')
	for p in people:
	    info.append((mov_title, p, won))


    return info


In [482]:
win_noms[0][1][0].xpath('./*')

[<Element h3 at 0x10fab28e8>,
 <Element div at 0x10fab5100>,
 <Element h3 at 0x10fbb3578>,
 <Element div at 0x10f70e788>,
 <Element hr at 0x10fbb35d0>,
 <Element div at 0x10f7d79f0>,
 <Element hr at 0x10fbb3628>,
 <Element div at 0x10f7d7ba8>,
 <Element hr at 0x10fbb3680>,
 <Element div at 0x10f7d7cb0>,
 <Element hr at 0x10fbb36d8>,
 <Element div at 0x10f7d7db8>,
 <Element hr at 0x10fbb3730>,
 <Element div at 0x10f7d7e10>,
 <Element hr at 0x10fbb3788>,
 <Element div at 0x10f7d7fc8>]

In [483]:
get_win_nom_data(win_noms[0][1][0])

[('Birdman or (The Unexpected Virtue of Ignorance)',
  u'Alejandro G. I\xf1\xe1rritu'),
 ('Birdman or (The Unexpected Virtue of Ignorance)', 'John Lesher'),
 ('Birdman or (The Unexpected Virtue of Ignorance)', 'James W. Skotchdopole'),
 ('American Sniper', 'Clint Eastwood'),
 ('American Sniper', 'Robert Lorenz'),
 ('American Sniper', 'Andrew Lazar'),
 ('American Sniper', 'Bradley Cooper'),
 ('American Sniper', 'Peter Morgan'),
 ('Boyhood', 'Richard Linklater'),
 ('Boyhood', 'Cathleen Sutherland'),
 ('The Imitation Game', 'Nora Grossman'),
 ('The Imitation Game', 'Ido Ostrowsky'),
 ('The Imitation Game', 'Teddy Schwarzman'),
 ('The Grand Budapest Hotel', 'Wes Anderson'),
 ('The Grand Budapest Hotel', 'Scott Rudin'),
 ('The Grand Budapest Hotel', 'Steven Rales'),
 ('The Grand Budapest Hotel', 'Jeremy Dawson'),
 ('Selma', 'Christian Colson'),
 ('Selma', 'Oprah Winfrey'),
 ('Selma', 'Dede Gardner'),
 ('Selma', 'Jeremy Kleiner'),
 ('The Theory of Everything', 'Tim Bevan'),
 ('The Theory of 